In [13]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras import metrics as m
from numpy import zeros
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.python.keras.layers import Bidirectional, GlobalMaxPool1D,Bidirectional, GlobalMaxPool2D, SpatialDropout1D, concatenate, Conv1D
from tensorflow.python.keras.models import Model
from tensorflow.python.keras import initializers, regularizers, constraints, optimizers, layers
import matplotlib.pyplot as plt
import gensim.models.keyedvectors as word2vec


In [14]:
from tensorflow.python.keras.layers import MaxPooling1D

In [15]:
import gc
import joblib
import pickle

In [20]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [21]:
train

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [22]:
train.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [23]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
list_classes

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [24]:
y = train[list_classes].values
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [25]:
train_sentences = train["comment_text"]
test_sentences = test["comment_text"]

In [26]:
train_sentences

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object

In [27]:
def clean_comment(comment):
    
    #  lower case  
    comment_lower_case = comment.lower()
    #remove or replace the unwanted expressions  
    comment_inter = re.sub(r"i'm", "i am ", comment_lower_case)
    comment_inter = re.sub(r"what's", "what is ", comment_inter)
    comment_inter = re.sub(r"\'ve", " have ", comment_inter)
    comment_inter = re.sub(r"\'s", " ", comment_inter)
    comment_inter = re.sub(r"can't", "can not ", comment_inter)
    comment_inter = re.sub(r"n't", " not ", comment_inter)
    comment_inter = re.sub(r"\'re", " are ", comment_inter)
    comment_inter = re.sub(r"\'d", " would ", comment_inter)
    comment_inter = re.sub(r"\'ll", " will ", comment_inter)
    comment_inter = re.sub(r"\'scuse", " excuse ", comment_inter)
    comment_inter = re.sub('\W', ' ', comment_inter)
    comment_inter = re.sub('\s+', ' ', comment_inter)
    # Remove punctuations and numbers
    cleaned_comment = re.sub('[^a-zA-Z]', ' ', comment_inter)
    # Single character removal
    cleaned_comment = re.sub(r"\s+[a-zA-Z]\s+", ' ', cleaned_comment)
    return cleaned_comment

In [28]:
train_sentences = train_sentences.apply(lambda x : clean_comment(x))
test_sentences = test_sentences.apply(lambda x : clean_comment(x))

In [ ]:
train_sentences.head()

In [30]:
X_train, X_test, Y_train, Y_valid = train_test_split(train_sentences,y,test_size=0.3,random_state=0)

In [31]:
X_train.shape

(111699,)

In [32]:
Y_train.shape

(111699, 6)

In [33]:
# prepare tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(X_train))

In [34]:
# integer encode the document
list_tokenized_train = tokenizer.texts_to_sequences(X_train) # Transforms each text in X_train to a sequence of integers.
list_tokenized_valid = tokenizer.texts_to_sequences(X_test) # Transforms each text in X_train to a sequence of integers.

In [35]:
word_index = tokenizer.word_index # A dictionary of words and their uniquely assigned integers.

In [36]:
word2index = tokenizer.word_index

In [37]:
joblib.dump(tokenizer, 'tokenizer.pkl')


['tokenizer.pkl']

In [38]:
def get_max_len():
    comment_len = []
    for comment in X_train:
        comment_len.append(len(comment))
    return int(np.ceil(np.mean(comment_len)))

In [39]:
maxlen = get_max_len()

In [40]:
X_train_padded = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test_padded = pad_sequences(list_tokenized_valid, maxlen=maxlen)

In [41]:
X_train_padded.shape

(111699, 373)

In [42]:
X_test_padded.shape

(47872, 373)

### Using the 300 dimensional GLove Word Embeddings
(the dimension has no special meaning, it is a hyperparameter, chosen by the creators of GloVe.)

In [43]:
embeddings_dictionary_glove = dict()  
path = 'glove.6B.300d.txt'
glove_file = open(path, encoding="utf8")

for line in glove_file: 
    records = line.split()  
    word = records[0]  
    vector_dimensions = np.asarray(records[1:], dtype='float32')  
    embeddings_dictionary_glove[word] = vector_dimensions  
glove_file.close()

we need to create a matrix of one embedding for each word in the training dataset

We enumerate all unique words in the Tokenizer.word_index and locate the embedding weight vector from the loaded GloVe embedding.

The result is a matrix of weights only for words we will see during training.

In [44]:
#create embedding matrix
embedding_matrix_glove = zeros((len(tokenizer.word_index)+1, 300))

# searching for vactors in Glove
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary_glove.get(word)
    if embedding_vector is not None:
        embedding_matrix_glove[index] = embedding_vector
        # words not found in embedding index will be all-zeros.

In [45]:
print('Found %s word vectors.' % len(embedding_matrix_glove))

Found 137611 word vectors.


word vectors are vectors of numbers that represent the meaning of a word.

In [46]:
embedding_matrix_glove.shape[1]

300

### Model

In [47]:
embedding_matrix_glove.shape[1]

300

In [48]:
word2index = tokenizer.word_index

The key difference is that the embedding layer can be seeded with the GloVe word embedding weights. We chose the 300-dimensional version, therefore the Embedding layer must be defined with output_dim set to 300. Finally, we do not want to update the learned word weights in this model, therefore we will set the trainable attribute for the model to be False.

In [49]:
from keras.models import Sequential
from tensorflow.keras.layers import InputLayer


In [50]:
model = Sequential() #with GloVe embeddings
model.add(InputLayer(input_shape=(maxlen, )))
model.add(Embedding(len(word2index)+1, 
                    300, 
                    weights=[embedding_matrix_glove],
                    input_length = maxlen,
                    trainable=False))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(128))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.1)) 
model.add(Dense(6, activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 373, 300)          41283300  
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 373, 300)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 128)               219648    
_________________________________________________________________
dense (Dense)                (None, 50)                6450      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 306       
Total params: 41,509,704
Trainable params: 226,404
Non-trainable params: 41,283,300
______________________________________

In [ ]:
batch_size = 128
epochs = 10
model.fit(X_train_padded,Y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=(X_test_padded, Y_valid),
          callbacks=EarlyStopping(monitor='val_loss'))

Epoch 1/10
873/873 [==============================] - 888s 1s/step - loss: 0.0779 - accuracy: 0.8879 - val_loss: 0.0530 - val_accuracy: 0.9938
Epoch 2/10
642/873 [=====================>........] - ETA: 3:34 - loss: 0.0543 - accuracy: 0.9618

In [ ]:
print("ROC AUC Score  for LSTM model using GloVe Embedding : %0.3f" %model.evaluate(X_test_padded, Y_valid)[1])
